In [2]:
import torch
import torchvision
from torchvision.models.detection.transform import GeneralizedRCNNTransform
from torchvision.models.detection.backbone_utils import mobilenet_backbone

from torchvision.models.detection.anchor_utils import AnchorGenerator
nn = torch.nn
import os
import numpy as np
from src.utils.audio_utils import compute_spectrogram, load_audio_file
from src.utils import path_utils


/Users/paulbp_dty/Desktop/P3_BBF/audio_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [58]:
class OneShotDataset(torch.utils.data.Dataset):
    def __init__(self, detection_dir,transform_audio, device):
        self.detection_dir = detection_dir
        self.device = device
        self.samplepath = os.path.join(self.detection_dir, 'samples')
        self.targetpath = os.path.join(self.detection_dir, 'target')
        self.indexes = list(map(lambda x: x.split('.')[0],os.listdir(self.samplepath)))
        self.transform_audio = transform_audio
        self.transform_image = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
        self.spec_hight=120

    def __len__(self):
        return len(os.listdir((self.samplepath)))
        
    def __getitem__(self,idx):
        index=self.indexes[idx]
        data,fs = load_audio_file(os.path.join(self.samplepath,index +'.wav'))
        targets = np.load(os.path.join(self.targetpath,index +'.npy'))
        data = self.transform_audio(data)
        boxes=torch.as_tensor([[start,0,end,data.shape[0]] for [_,start,end]in targets], dtype=torch.float32)
        labels=torch.as_tensor([target[0] for target in targets], dtype=torch.int32)
        #target_dict={'boxes':boxes,'labels':labels}
        # targets=[{'boxes':torch.as_tensor([start,0,end,data.shape[0]], dtype=torch.float32),'labels':lbl} for [lbl,start,end]in targets]
        x = self.transform_image(data)
        return x, boxes, labels

In [70]:
# from torch.nn.utils.rnn import pad_sequence
# def transform_audio(data):
#     _, _, specto = compute_spectrogram(data, 24000, nperseg=256, noverlap=256/2, scale="dB")
#     # freq clip
#     specto = specto[:120, :]
#     return specto

# def padding_collate(batch):
#     nbatch, image_list, boxes_list, labels_list = [], [], [], []
#     for data in batch:
#         image_list.append(data[0])
#         boxes_list.append(data[1])
#         labels_list.append(data[2])
#     boxes_list=pad_sequence(boxes_list, batch_first=True)
#     labels_list=pad_sequence(labels_list, batch_first=True)
#     for i in range(len(image_list)):
#         nbatch.append((image_list[i], {'boxes':boxes_list[i], 'labels':labels_list[i]}))
#     return nbatch

# detection_dataloader = torch.utils.data.DataLoader(OneShotDataset(detection_dir=path_utils.get_detection_data_path(),
#                                                     transform_audio=transform_audio, device=device),collate_fn=padding_collate,batch_size=16)
# for data in detection_dataloader:
#     break

[(tensor([[[ -48.4338,  -48.0510,  -46.7459,  ...,  -44.4542,  -46.1067,
           -47.8241],
         [ -56.6873,  -44.9814,  -46.9355,  ...,  -39.1955,  -46.6519,
           -50.3317],
         [ -46.3867,  -54.8088,  -59.3137,  ...,  -43.2027,  -63.1286,
           -50.2026],
         ...,
         [ -97.3052,  -91.5861,  -84.6041,  ...,  -89.6452,  -93.3256,
           -85.4067],
         [ -94.4468,  -90.0997,  -84.4939,  ...,  -87.5063,  -98.0471,
           -88.4591],
         [ -89.2727,  -97.7166, -103.3719,  ...,  -95.3397,  -95.2801,
           -93.7884]]]), {'boxes': tensor([[2.7266e+04, 0.0000e+00, 4.3424e+04, 1.2000e+02],
        [4.6030e+03, 0.0000e+00, 3.4288e+04, 1.2000e+02],
        [3.8040e+03, 0.0000e+00, 3.2598e+04, 1.2000e+02],
        [1.7443e+04, 0.0000e+00, 4.0532e+04, 1.2000e+02],
        [1.4340e+03, 0.0000e+00, 4.7866e+04, 1.2000e+02],
        [3.7460e+03, 0.0000e+00, 4.2519e+04, 1.2000e+02],
        [3.3000e+01, 0.0000e+00, 4.5554e+04, 1.2000e+02],
       

In [51]:

# def rcnn_pretrained_backbone(num_classes : int, anchor_sizes : tuple, aspect_ratios : tuple, parameters : dict = {}):
#     """
#     Return a faster rcnn with a mobilenetv3 backbone.

#     Args:
#         num_classes (int): Number of classes expected to return (background should be taken into account)
#         parameters (dict, optional): Dictionnary for the different following parameters. Defaults to {}.

#     Returns:
#         (torchvision.models.detection.faster_rcnn.FasterRCNN): Model implementation of pytorch

#     >>> training
#     # RPN
#     rpn_pre_nms_top_n_train (int): number of proposals to keep before applying NMS during training
#     rpn_post_nms_top_n_train (int): number of proposals to keep after applying NMS during training
#     rpn_fg_iou_thresh (float): minimum IoU between the anchor and the GT box so that they can be
#         considered as positive during training of the RPN.
#     rpn_bg_iou_thresh (float): maximum IoU between the anchor and the GT box so that they can be
#         considered as negative during training of the RPN.
#     rpn_batch_size_per_image (int): number of anchors that are sampled during training of the RPN
#         for computing the loss
#     rpn_positive_fraction (float): proportion of positive anchors in a mini-batch during training
#         of the RPN

#     # Head
#     box_fg_iou_thresh (float): minimum IoU between the proposals and the GT box so that they can be
#         considered as positive during training of the classification head
#     box_bg_iou_thresh (float): maximum IoU between the proposals and the GT box so that they can be
#         considered as negative during training of the classification head
#     box_batch_size_per_image (int): number of proposals that are sampled during training of the
#         classification head
#     box_positive_fraction (float): proportion of positive proposals in a mini-batch during training
#         of the classification head



#     >>> inference
#     # RPN
#     rpn_pre_nms_top_n_test (int): number of proposals to keep before applying NMS during testing
#     rpn_post_nms_top_n_test (int): number of proposals to keep after applying NMS during testing
#     rpn_score_thresh (float): during inference, only return proposals with a classification score
#         greater than rpn_score_thresh
#     rpn_nms_thresh (float): NMS threshold used for postprocessing the RPN proposals

#     # Head
#     box_score_thresh (float): during inference, only return proposals with a classification score
#         greater than box_score_thresh

#     box_nms_thresh (float): NMS threshold for the prediction head. Used during inference
#     box_detections_per_img (int): maximum number of detections per image, for all classes.

#     """

#     pretrained_backbone=True
#     trainable_backbone_layers=6 # All backbone is trainable

#     backbone = mobilenet_backbone("mobilenet_v3_large", pretrained_backbone, True, trainable_layers=trainable_backbone_layers)

#     model = torchvision.models.detection.faster_rcnn.FasterRCNN(backbone,
#                                                                 num_classes,
#                                                                 rpn_anchor_generator=AnchorGenerator(anchor_sizes, aspect_ratios),
#                                                                 **parameters)

#     # Custom transform ie no transform (only postprocessing)
#     model.transform = GeneralizedRCNNTransform(min_size=489, max_size=2000, image_mean=[0, 0, 0], image_std=[1, 1, 1])
#     return model

In [7]:
# def rcnn_pretrained_backbone_train(num_classes : int, anchor_sizes : tuple, aspect_ratios : tuple, parameters : dict = {}):
#     """
#     Return a jit compiled faster rcnn with a mobilenetv3 backbone.
#     """
#     model = rcnn_pretrained_backbone(num_classes, anchor_sizes, aspect_ratios, parameters)
#     model._has_warned = True # Remove warning about "RCNN always returns a (Losses, Detections) tuple in scripting"
#     return torch.jit.script(model)

In [8]:
def loss_faster_rcnn(dict_losses : dict, training_rpn : bool, training_head : bool):
    """ Reduce the dictionnary of losses

    Args:
        dict_losses (dict): Dictionnary of losses
        training_rpn (bool): Bool to train the rpn
        training_head (bool): Bool to train the head

    Returns:
        (torch.Tensor): Loss
    """
    # Dict("loss_classifier", "loss_box_reg", "loss_objectness", "loss_rpn_box_reg")
    loss = torch.zeros((), dtype=torch.float32, device=device)
    if training_rpn:
        loss += dict_losses["loss_objectness"]
        loss += dict_losses["loss_rpn_box_reg"]
    if training_head:
        loss += dict_losses["loss_classifier"]
        loss += dict_losses["loss_box_reg"]

In [9]:
def average_losses(dataloader, mean_loss):
    mean_loss["loss_objectness"] /= len(dataloader)
    mean_loss["loss_rpn_box_reg"] /= len(dataloader)
    mean_loss["loss_classifier"] /= len(dataloader)
    mean_loss["loss_box_reg"] /= len(dataloader)
    return mean_loss

def accumulate_losses(mean_loss, losses):
    mean_loss["loss_objectness"] += losses["loss_objectness"].item()
    mean_loss["loss_rpn_box_reg"] += losses["loss_rpn_box_reg"].item()
    mean_loss["loss_classifier"] += losses["loss_classifier"].item()
    mean_loss["loss_box_reg"] += losses["loss_box_reg"].item()

In [81]:
from torch.nn.utils.rnn import pad_sequence
def transform_audio(data):
    _, _, specto = compute_spectrogram(data, 24000, nperseg=256, noverlap=256/2, scale="dB")
    # freq clip
    specto = specto[:120, :]
    return specto

def padding_collate(batch):
    targets, image_list, boxes_list, labels_list = [], [], [], []
    for data in batch:
        image_list.append(data[0])
        boxes_list.append(data[1])
        labels_list.append(data[2])
    boxes_list=pad_sequence(boxes_list, batch_first=True)
    labels_list=pad_sequence(labels_list, batch_first=True)
    for i in range(len(boxes_list)):
        targets.append(( {'boxes':boxes_list[i], 'labels':labels_list[i]}))
    return image_list, targets

detection_dataloader = torch.utils.data.DataLoader(OneShotDataset(detection_dir=path_utils.get_detection_data_path(),
                                                    transform_audio=transform_audio, device=device),collate_fn=padding_collate,batch_size=2)

In [82]:
backbone = mobilenet_backbone("mobilenet_v3_large",True, False, trainable_layers=6)

model = torchvision.models.detection.faster_rcnn.FasterRCNN(backbone,
                                                                num_classes=2,
                                                                rpn_anchor_generator=AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),)))

model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
sgd_optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(sgd_optimizer,
                                               step_size=3,
                                               gamma=0.1)
                                        
loss_fn_frrcnn=lambda x:loss_faster_rcnn(x,True,True)

In [85]:
def train_loop(dataloader, model, loss_fn, optimizer, scheduler, macro_batch=1):
    # Initialize training
    model.train()
    optimizer.zero_grad()

    mean_loss = {"loss_objectness": 0., "loss_rpn_box_reg": 0., "loss_classifier": 0., "loss_box_reg": 0.}

    # Iterate over the dataset
    for batch, (X, targets) in enumerate(dataloader):
        print(targets)
        # Work with the GPU if available
        X = list(x.to(device) for x in X)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        # Compute prediction error
        losses, detections = model(X, targets)
        accumulate_losses(mean_loss, losses)
        loss = loss_fn(losses)

        # Backpropagation
        loss.backward()
        if (batch+1) % macro_batch == 0 or batch == len(dataloader) - 1:
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()

        # Print metrics
        if batch % 30 == 0 or batch == len(dataloader) - 1:
            loss_value, current = loss.item(), (batch+1) * len(X)
        print(loss_value)
    return average_losses(dataloader, mean_loss)

In [86]:
train_loop(dataloader=detection_dataloader, model=model, loss_fn=loss_fn_frrcnn, optimizer=sgd_optimizer, scheduler=lr_scheduler, macro_batch=1)


[{'boxes': tensor([[2.7266e+04, 0.0000e+00, 4.3424e+04, 1.2000e+02],
        [4.6030e+03, 0.0000e+00, 3.4288e+04, 1.2000e+02],
        [3.8040e+03, 0.0000e+00, 3.2598e+04, 1.2000e+02],
        [1.7443e+04, 0.0000e+00, 4.0532e+04, 1.2000e+02],
        [1.4340e+03, 0.0000e+00, 4.7866e+04, 1.2000e+02],
        [3.7460e+03, 0.0000e+00, 4.2519e+04, 1.2000e+02],
        [3.3000e+01, 0.0000e+00, 4.5554e+04, 1.2000e+02],
        [1.3210e+03, 0.0000e+00, 4.5577e+04, 1.2000e+02],
        [5.5680e+03, 0.0000e+00, 4.7548e+04, 1.2000e+02],
        [9.5400e+02, 0.0000e+00, 4.6437e+04, 1.2000e+02],
        [4.0010e+03, 0.0000e+00, 4.7878e+04, 1.2000e+02],
        [3.0550e+03, 0.0000e+00, 4.6864e+04, 1.2000e+02],
        [5.0000e+00, 0.0000e+00, 4.7919e+04, 1.2000e+02],
        [2.9250e+03, 0.0000e+00, 4.6390e+04, 1.2000e+02],
        [3.6800e+02, 0.0000e+00, 4.7444e+04, 1.2000e+02],
        [6.2850e+03, 0.0000e+00, 4.6449e+04, 1.2000e+02],
        [5.1960e+03, 0.0000e+00, 4.7987e+04, 1.2000e+02],
   

ValueError: All bounding boxes should have positive height and width. Found invalid box [0.0, 0.0, 0.0, 0.0] for target at index 0.